## Lexicase Selection注意事项

对于Lexicase Selection，适应度评估需要更改为返回多个误差组成的向量，而不是均方误差（MSE）。这样，Lexicase Selection才能独立考虑每个个体在每个测试样本上的表现，从而提高选择的多样性。

In [7]:
import numpy as np
import math
import operator

from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    
    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100) 
    
    return tuple((func(x) - x**2)**2)


# 创建个体和适应度函数，适应度数组大小与数据量相同
creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * 100)  # 假设我们有20个数据点
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

### 遗传算子
选择算子需要改成Lexicase Selection，其他不需要改变。对于回归问题，需要使用AutomaticEpsilonLexicase。而对于分类问题，则使用Lexicase即可。

In [8]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selAutomaticEpsilonLexicase)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

In [9]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
population = toolbox.population(n=20)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=20, stats=mstats, halloffame=hof, verbose=True)
print(str(hof[0]))


   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max  	min	nevals	std    	avg 	gen	max	min	nevals	std    
0  	20    	1976.63	0  	12544	0  	20    	2791.12	4.65	0  	7  	3  	20    	1.58981
1  	12    	313.902	1  	12100	0  	12    	1323.29	3.5 	1  	7  	3  	12    	1.07238
2  	11    	1045.82	2  	40000	0  	11    	3494.81	3.9 	2  	9  	3  	11    	1.64012
3  	10    	7999.5 	3  	1.21e+06	0  	10    	67582.2	3.65	3  	8  	3  	10    	1.4239 
4  	10    	7897.13	4  	1.1881e+06	0  	10    	66124.7	3.5 	4  	8  	3  	10    	1.20416
5  	12    	211.535	5  	12100     	0  	12    	1097.11	3.6 	5  	9  	2  	12    	1.71464
6  	11    	104.067	6  	10000     	0  	11    	768.451	3.2 	6  	5  	3  	11    	0.6    
7  	11    	31682.4	7  	5.29e+06  	0  	11    	281300 	3.75	7  	9  	3  	11    	1.51245
8  	13    	418.07 	8  	12100     	0  	13   